In [22]:
import pandas as pd
from sklearn.metrics import classification_report
from openai import OpenAI
from transformers import pipeline
import os
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.getenv("API_KEY_OPENAI"))

In [23]:
df = pd.read_csv('INPUT/emociones.csv')
df = df[['opinion', 'label']]
df = df.dropna()
df = df.sample(30, random_state=42).reset_index(drop=True)
print(df)


                       opinion     label
0          Es un gran servicio  positiva
1                   Es regular   neutral
2          Es un producto útil  positiva
3                 Es un fraude  negativa
4               Me gusta mucho  positiva
5               Es un desastre  negativa
6           Es un mal servicio  negativa
7        Es un producto inútil  negativa
8           No lo recomendaría  negativa
9       Me encantó el producto  positiva
10                 No me gusta  negativa
11               Es fantástico  positiva
12                  Me encanta  positiva
13         Es un buen producto  positiva
14                Es muy bueno  positiva
15          Es un mal servicio  negativa
16    Es una pérdida de dinero  negativa
17         No es nada especial   neutral
18       Es un producto normal   neutral
19                     Te odio  negativa
20         Es un buen servicio  positiva
21         Es un gran producto  positiva
22      Es un producto decente   neutral
23  Es un produc

In [24]:
zero_shot_prompt = """Clasifica el sentimiento del siguiente texto como positivo, negativo o neutral:

"{}"
"""

few_shot_prompt = """Clasifica el sentimiento del siguiente texto como positivo, negativo o neutral.
Ejemplo 1: \"Me encantó la película\" → positivo
Ejemplo 2: \"No me gustó para nada\" → negativo
Ejemplo 3: \"Está bien, sin más\" → neutral
Texto: "{}"
Respuesta:"""

cot_prompt = """Analiza el siguiente texto y razona si es positivo, negativo o neutral.
Texto: "{}"
Primero, explica por qué. Luego, indica si es positivo, negativo o neutral."""

prompts = {
    "zero-shot": zero_shot_prompt,
    "few-shot": few_shot_prompt,
    "chain-of-thought": cot_prompt
}

In [25]:

def predict_openai(opinion, prompt_template):
    prompt = prompt_template.format(opinion)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return response.choices[0].message["content"].strip().lower()

results = {}

for method, prompt_template in prompts.items():
    predictions = []
    for opinion in df['opinion']:
        pred = predict_openai(opinion,prompt_template)
        pred = pred.lower()
        if "positivo" in pred:
            predictions.append("positiva")
        elif "negativo" in pred:
            predictions.append("negativa")
        elif "neutral" in pred:
            predictions.append("neutral")
        else:
            predictions.append("desconocido")  # Fallback
    results[method] = predictions

for method, preds in results.items():
    print(f"\n=== {method.upper()} ===")
    print(classification_report(df['label'], preds, zero_division=0))


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}